In [1]:
import os
from diffusers import DiffusionPipeline, StableDiffusionPipeline,StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

2024-08-06 14:03:57.342886: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [7]:
# make sure you're logged in with `huggingface-cli login`
from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]

image.save("results/astronaut_rides_horse.png")

Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/runwayml/stable-diffusion-v1-5 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1b958ec280>: Failed to establish a new connection: [Errno 111] Connection refused')))"), '(Request ID: efc4df79-b6b6-4a96-94d0-9061e384dabd)').
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
pipe = DiffusionPipeline.from_pretrained(pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
# pipe = StableDiffusionPipeline("stabilityai/stable-diffusion-xl-base-1.0").to("cuda")
pipe.to("cuda")

Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/stabilityai/stable-diffusion-xl-base-1.0 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1b9d07a2e0>: Failed to establish a new connection: [Errno 111] Connection refused')))"), '(Request ID: 53732b46-7627-4f96-a1a6-0c142e886310)').
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.29.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
prompt = "Create an image of a tall cactus in a desert. The cactus has multiple arms, is green, and covered in small spines. The desert features sandy terrain, a few rocks, sparse vegetation, and a bright blue sky with a few clouds."
for step in [1,5,10,20,30,50,80,100,200,300,500,1000]:
    img = pipe(prompt=prompt,num_inference_steps=step).images[0]
    img.save(f"results/cactus_base_s{step}.png")

In [ ]:
from PIL import Image
image_folder = "results"
image_paths = [f"{image_folder}/sunflower_base_s{step}.png" for step in [1,5,10,20,30,50,80,100,200,300,500,1000]]
grid_size = (3, 4)

assert len(image_paths) == grid_size[0] * grid_size[1], "Number of images does not match grid size"

# Open the first image to get the size
first_img = Image.open(image_paths[0])
img_width, img_height = first_img.size

# Create a new image with the appropriate size
grid_img = Image.new('RGB', (img_width * grid_size[1], img_height * grid_size[0]))

# Paste each image into the grid
for idx, img_path in enumerate(image_paths):
    img = Image.open(img_path)
    x = (idx % grid_size[1]) * img_width
    y = (idx // grid_size[1]) * img_height
    grid_img.paste(img, (x, y))

# Save the final grid image
grid_img.save(f"{image_folder}/cactus_grid.png")

In [ ]:
# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 200
high_noise_frac = 0.8

prompt = "real sunflower"

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

image.save("results/sunflower_refiner.png")